In [4]:
from sqlalchemy import create_engine, func, desc
from sqlalchemy.orm import sessionmaker

from sqlalchemy import BigInteger, Column, ForeignKey, Float, Integer, String
from sqlalchemy.orm import declarative_base

# Establishing a connection to the database

In [10]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="movielens_db",
    user="postgres",
    password="postgres",
    host="db",
    port="5432"
)

# Testing of DB queries

## 1. How many movies are in the dataset?

In [11]:
# Create a cursor object
cur = conn.cursor()

cur.execute("SELECT COUNT(*) FROM movies;")
movie_count = cur.fetchone()[0]
print(f"Total number of movies: {movie_count}")

Total number of movies: 9742


## 2. What is the most common genre of movie?

In [12]:
cur.execute("""
    SELECT genres, COUNT(*)
    FROM movies
    GROUP BY genres
    ORDER BY COUNT(*) DESC
    LIMIT 1;
""")
most_common_genre = cur.fetchone()[0]
print(f"Most common genre: {most_common_genre}")

Most common genre: Drama


## 3. What are top 10 movies with the highest rating?

In [13]:
cur.execute("""
    SELECT m.title, AVG(r.rating) as avg_rating
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY m.title
    ORDER BY avg_rating DESC
    LIMIT 10;
""")
top_movies = cur.fetchall()
print("Top 10 movies with the highest average rating:")
for movie in top_movies:
    print(movie)

Top 10 movies with the highest average rating:
('Brother (Brat) (1997)', 5.0)
('Eva (2011)', 5.0)
('Go for Zucker! (Alles auf Zucker!) (2004)', 5.0)
('Holy Motors (2012)', 5.0)
('Connections (1978)', 5.0)
('Watching the Detectives (2007)', 5.0)
('Girls About Town (1931)', 5.0)
('Story of Women (Affaire de femmes, Une) (1988)', 5.0)
('Nasu: Summer in Andalusia (2003)', 5.0)
('Into the Woods (1991)', 5.0)


## 4. Who are the 5 most often rating users?

In [14]:
cur.execute("""
    SELECT userId, COUNT(*) as rating_count
    FROM ratings
    GROUP BY userId
    ORDER BY rating_count DESC
    LIMIT 5;
""")
most_active_users = cur.fetchall()
print("Top 5 most active users:")
for user in most_active_users:
    print(user)

Top 5 most active users:
(414, 2698)
(599, 2478)
(474, 2108)
(448, 1864)
(274, 1346)


## 5. When was done first and last rate included in dataset, and what was the rated movie title?

In [16]:
cur.execute("""
    SELECT m.title, to_timestamp(r.timestamp) as rating_time
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    ORDER BY r.timestamp ASC
    LIMIT 1;
""")
first_rating = cur.fetchone()
print(f"First rating: {first_rating[0]} at {first_rating[1]}")

cur.execute("""
    SELECT m.title, to_timestamp(r.timestamp) as rating_time
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    ORDER BY r.timestamp DESC
    LIMIT 1;
""")
last_rating = cur.fetchone()
print(f"Last rating: {last_rating[0]} at {last_rating[1]}")

First rating: Copycat (1995) at 1996-03-29 18:36:55+00:00
Last rating: Crumb (1994) at 2018-09-24 14:27:30+00:00


## 6. Find all movies released in 1990.

In [17]:
cur.execute("""
    SELECT title 
    FROM movies
    WHERE title LIKE '%(1990)%';
""")
movies_1990 = cur.fetchall()
print("Movies released in 1990:")
for movie in movies_1990:
    print(movie[0])

Movies released in 1990:
Home Alone (1990)
Ghost (1990)
Dances with Wolves (1990)
Pretty Woman (1990)
Days of Thunder (1990)
Grifters, The (1990)
Tie Me Up! Tie Me Down! (¡Átame!) (1990)
Paris Is Burning (1990)
Goodfellas (1990)
Trust (1990)
Rosencrantz and Guildenstern Are Dead (1990)
Miller's Crossing (1990)
Femme Nikita, La (Nikita) (1990)
Pump Up the Volume (1990)
Cyrano de Bergerac (1990)
Amityville Curse, The (1990)
Die Hard 2 (1990)
Young Guns II (1990)
Marked for Death (1990)
Hunt for Red October, The (1990)
King of New York (1990)
Metropolitan (1990)
Child's Play 2 (1990)
Exorcist III, The (1990)
Gremlins 2: The New Batch (1990)
Back to the Future Part III (1990)
Godfather: Part III, The (1990)
Rescuers Down Under, The (1990)
NeverEnding Story II: The Next Chapter, The (1990)
My Blue Heaven (1990)
Sheltering Sky, The (1990)
Edward Scissorhands (1990)
Tales from the Darkside: The Movie (1990)
Heart Condition (1990)
Rocky V (1990)
Dick Tracy (1990)
Arachnophobia (1990)
Problem C

# ORM implementation

In [ ]:
Base = declarative_base()

class Movie(Base):
    __tablename__ = 'movies'

    movieId = Column(Integer, primary_key = True, autoincrement = True)
    title = Column(String)
    genres = Column(String)

class Rating(Base):
    __tablename__ = 'ratings'

    ratingid = Column(Integer, primary_key = True, autoincrement = True)
    userid = Column(Integer)
    movieid = Column(Integer, ForeignKey('movies.movieid'))
    rating = Column(Float)
    timestamp = Column(BigInteger)

